In [1]:
import os
import sys
from functools import partial

import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow_datasets as tfds

import sklearn
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def load_tf_dataset(name='mnist'):
    (train_ds, validation_ds, test_ds), ds_info = tfds.load(
        name,
        split=[
              'train[:70%]',
              'train[70%:]',
              'test'
        ],
        shuffle_files=True,
        as_supervised=True,
        with_info=True,
    )
    train_ds_size = tf.data.experimental.cardinality(train_ds).numpy()
    test_ds_size = tf.data.experimental.cardinality(test_ds).numpy()
    validation_ds_size = tf.data.experimental.cardinality(validation_ds).numpy()
        
    def process_images(image, label):
        image = tf.image.per_image_standardization(image)
        image = tf.image.resize(image, (227, 227))
        return image, label


    train_ds = (train_ds
                      .map(process_images)
                      .shuffle(buffer_size=train_ds_size)
                      .batch(batch_size=32, drop_remainder=True))
    test_ds = (test_ds
                      .map(process_images)
                      .shuffle(buffer_size=train_ds_size)
                      .batch(batch_size=32, drop_remainder=True))
    validation_ds = (validation_ds
                      .map(process_images)
                      .shuffle(buffer_size=train_ds_size)
                      .batch(batch_size=32, drop_remainder=True))
    
    return (train_ds, validation_ds, test_ds), ds_info

In [3]:
(ds_train, ds_valid, ds_test), ds_info = load_tf_dataset('cifar10')

In [ ]:
tfds.show_examples(ds_train, ds_info)

## AlexNet

In [197]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(32, 32, 3)),
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),
    tf.keras.layers.Conv2D(32, (5, 5), strides=(1, 1), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),
    tf.keras.layers.Conv2D(64, (5, 5), strides=(1, 1), activation='relu', padding='same'),
    tf.keras.layers.Conv2D(64, (5, 5), strides=(1, 1), activation='relu', padding='same'),
    tf.keras.layers.Conv2D(32, (5, 5), strides=(1, 1), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='softmax'), 
])

In [198]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=tf.keras.optimizers.SGD(lr=0.001),
              metrics=['acc'])

## GoogleLeNet

In [4]:
class Inception(tf.keras.layers.Layer):
    def __init__(self, num_filter_1, num_filter_2, num_filter_3, \
                     num_filter_4, num_filter_5, num_filter_6, **kwargs):
        super(Inception, self).__init__(**kwargs)
        self.conv1_1 = tf.keras.layers.Conv2D(num_filter_1, (1, 1), strides=1, \
                                              activation='relu', padding='SAME')
        self.conv1_2 = tf.keras.layers.Conv2D(num_filter_2, (1, 1), strides=1, \
                                              activation='relu', padding='SAME')
        self.maxpool1 = tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(1, 1), padding='SAME')
        self.conv2_1 = tf.keras.layers.Conv2D(num_filter_3, (1, 1), strides=1, \
                                              activation='relu', padding='SAME')
        self.conv2_2 = tf.keras.layers.Conv2D(num_filter_4, (3, 3), strides=1, \
                                              activation='relu', padding='SAME')
        self.conv2_3 = tf.keras.layers.Conv2D(num_filter_5, (5, 5), strides=1, \
                                              activation='relu', padding='SAME')
        self.conv2_4 = tf.keras.layers.Conv2D(num_filter_6, (1, 1), strides=1, \
                                              activation='relu', padding='SAME')
    
    def call(self, inputs):
        conv1_1 = self.conv1_1(inputs)
        conv1_2 = self.conv1_2(inputs)
        maxpool1 = self.maxpool1(inputs)
        conv2_1 = self.conv2_1(inputs)
        conv2_2 = self.conv2_2(conv1_1)
        conv2_3 = self.conv2_3(conv1_2)
        conv2_4 = self.conv2_4(maxpool1)
        output = tf.keras.layers.concatenate([conv2_1, conv2_2, conv2_3, conv2_4])
        return output

In [5]:
Convolutional = partial(tf.keras.layers.Conv2D,
                        padding='SAME',
                        activation='relu')
MaxPooling = partial(tf.keras.layers.MaxPool2D,
                        pool_size=(3, 3),
                        strides=(2, 2),
                        padding='SAME')

In [6]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(227, 227, 3)),
    Convolutional(64, kernel_size=(7, 7), strides=(2, 2)),
    MaxPooling(),
    tf.keras.layers.BatchNormalization(),
    Convolutional(64, kernel_size=(1, 1), strides=(1, 1)),
    Convolutional(192, kernel_size=(3, 3), strides=(1, 1)),
    tf.keras.layers.BatchNormalization(),
    MaxPooling(),
    Inception(64, 128, 32, 32, 96, 16),
    Inception(128, 192, 96, 64, 128, 32),
    tf.keras.layers.AvgPool2D(pool_size=(3, 3)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(1000, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

In [7]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=tf.keras.optimizers.SGD(lr=0.001),
              metrics=['acc'])